## Loss
### BinaryCrossentropy / BinaryFocalCrossentropy

- 用于二元分类
- 默认from_logits=False，我们假设 y_pred 是概率（即 [0, 1] 中的值）。否则将 y_pred 解释为 logit 值的张量。

In [26]:
import keras
import tensorflow as tf

# from logits = True
y_true = [0, 1, 0, 0]
y_pred = [-18.6, 0.51, 2.94, -12.8]

l = keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True)
print(l)
l = keras.losses.binary_focal_crossentropy(y_true, y_pred, from_logits=True)
print(l)

# from logits = False
y_true = [[0, 1], [0, 0]]
y_pred = [[0.06, 0.9], [0.1, 0.1]]
loss = keras.losses.binary_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
print(loss)

loss = keras.losses.binary_focal_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
print(loss)


tf.Tensor(0.865458, shape=(), dtype=float32)
tf.Tensor(0.6912122, shape=(), dtype=float32)
tf.Tensor([0.08361798 0.10536055], shape=(2,), dtype=float32)
tf.Tensor([0.00063818 0.00105361], shape=(2,), dtype=float32)


### CategoricalCrossentropy / CategoricalFocalCrossentropy 

- 用于大于二的分类
- 默认from_logits=False，我们假设 y_pred 是概率（即 [0, 1] 中的值）。否则将 y_pred 解释为 logit 值的张量。
- 标签以 one_hot 表示形式提供

In [41]:
y_true = tf.constant([[0, 1, 0], [0, 0, 1]])
y_pred = tf.constant([[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
# Using 'auto'/'sum_over_batch_size' reduction type.
cce = keras.losses.CategoricalCrossentropy()
_cce1 = cce(y_true, y_pred)
print(_cce1)

cfe = keras.losses.CategoricalFocalCrossentropy()
print(cfe(y_true, y_pred))

tf.Tensor(1.1769392, shape=(), dtype=float32)
tf.Tensor(0.23315276, shape=(), dtype=float32)


### SparseCategoricalCrossentropy

- 用于大于二的分类
- 默认from_logits=False，我们假设 y_pred 是概率（即 [0, 1] 中的值）。否则将 y_pred 解释为 logit 值的张量。
- 标签以整数形式提供

In [42]:
y_true = tf.constant([1, 2])
#y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = keras.losses.SparseCategoricalCrossentropy()
_cce2 = scce(y_true, y_pred)
assert _cce1 == _cce2